In [112]:
from dotenv import load_dotenv

load_dotenv()

True

In [109]:
def get_option_probabilities(client, question, options):
    """Query the LLM to get probabilities for each option."""
    instruction = (
        "You are given a multiple-choice question. Assign a probability to each option based on correctness. "
        "Return a JSON object where the keys are option letters and values are probabilities."
    )

    options_text = "\n".join([f"{letter}: {text}" for letter, text in options.items()])
    input_text = f"Question: {question}\nOptions:\n{options_text}"

    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": input_text},
        ],
        response_format={"type": "json_object"},
    )

    probabilities = eval(response.choices[0].message.content)
    return probabilities


def poe_id2_elimination(client, question, options_list):
    """Implements PoE_ID² to eliminate two least likely options and select the best answer."""

    # Convert list of options into a dictionary {letter: value}
    options = {opt.split(": ")[0]: opt.split(": ")[1] for opt in options_list}

    probabilities = get_option_probabilities(client, question, options)

    # Sort options by probability
    sorted_options = sorted(probabilities.items(), key=lambda x: x[1])

    # Eliminate two options with the lowest probability
    remaining_options = {letter: options[letter] for letter, _ in sorted_options[1:]}

    # Recompute probabilities with the remaining options
    probabilities = get_option_probabilities(client, question, remaining_options)

    # Select the option with the highest probability
    final_answer = max(probabilities, key=probabilities.get)

    return final_answer


In [158]:
import re


def get_option_probabilities(client, question, options):
    """Query the LLM to get probabilities for each option."""
    instruction = """You are given a multiple-choice question. 
    For each round, identify the option that is least likely to be correct.  
    Continue until one option remains. Return a JSON object with the final probabilities for each option.
    Outputformat should be in ```json\n{{alphabet: probability}}``` brackets
    alphabet should in one of [A, B, C, D ,E, F]. 
    Note: Only give least likely with no more explaination and output final probability
    """

    options_text = "\n".join([f"{letter}: {text}" for letter, text in options.items()])
    input_text = f"Question: {question}\nOptions:\n{options_text}"

    # probabilities = eval(response.choices[0].message.content)
    response = client.responses.create(
        model="gpt-4o-mini",
        instructions=instruction,
        input=input_text,
    )
    print(response.output_text)
    match = re.search(r"```json\s*({.*?})\s*```", response.output_text, re.DOTALL)
    probabilities = match.group(1) if match else None
    return eval(probabilities)


def poe_id2_elimination(client, question, options_list):
    """Implements PoE_ID² to eliminate two least likely options and select the best answer."""

    # Convert list of options into a dictionary {letter: value}
    options = {opt.split(": ")[0]: opt.split(": ")[1] for opt in options_list}

    probabilities = get_option_probabilities(client, question, options)

    # Sort options by probability
    sorted_options = sorted(probabilities.items(), key=lambda x: x[1])

    # Eliminate two options with the lowest probability
    remaining_options = {letter: options[letter] for letter, _ in sorted_options[2:]}

    # Recompute probabilities with the remaining options
    probabilities = get_option_probabilities(client, question, remaining_options)

    # Select the option with the highest probability
    final_answer = max(probabilities, key=probabilities.get)

    return final_answer


In [167]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
question = """Question: Consider a link with packet loss probability of 0.2. What is the expected number of transmissions it would take to transfer 200 packets given that the stop and wait protocol is used?

"""
options = ["A: 125", "B: 250", "C: 225", "D: 150"]
# probabilities = get_option_probabilities(client, question, options)

answer = poe_id2_elimination(client, question, options)
print(f"Final answer: {answer}")


```json
{"A":0.0,"B":0.5,"C":0.2,"D":0.3}
```
```json
{"D": 0.2, "B": 0.8}
```
Final answer: B


In [168]:
ZERO_SHOT_TEMPLATE = """
The following are multiple choice questions (with answers) about software development.
Note: Alphabet should be one option in (A,B,C,D,E,F,G)
Question: {question}
{multiple_choices}
Answer in JSON format (in ```json``` brackets):
```json
{{"answer": "Your selected option here"}}
``` """
input_text = ZERO_SHOT_TEMPLATE.format(question=question, multiple_choices=options)
# probabilities = eval(response.choices[0].message.content)
response = client.responses.create(
    model="gpt-4o-mini",
    instructions="The answer selected from the multiple choice options should be one alphabet option (A,B,C,D,E,F).",
    input=input_text,
)
print(response.output_text)

```json
{"answer": "B"}
```
